In [ ]:
# 检测 导出的app.py 文件
file_path = "../app.py"
!flake8 {file_path} --max-line-length=240
!pylint {file_path}

In [ ]:
%%writefile ..\app.py
"""
项目初始化模板

这个文件用于设置项目的基本配置，包括:
1. 设置项目路径
2. 加载环境变量
3. 读取配置文件

注意: 此文件可以通过notebook导出到app.py，也可以直接在notebook中运行
"""
import os
import sys
import json
from dotenv import load_dotenv

# ===== 初始化项目路径 =====
# 根据运行环境(脚本或notebook)获取项目根目录
if "__file__" in globals():
    # 脚本环境: 使用文件的绝对路径
    current_dir = os.path.dirname(os.path.abspath(__file__))
    root_dir = os.path.normpath(os.path.join(current_dir, ".."))
else:
    # Jupyter Notebook环境: 使用当前工作目录
    current_dir = os.getcwd()
    current_dir = os.path.join(current_dir, "..")
    root_dir = os.path.normpath(os.path.join(current_dir))

# 规范化路径并添加到系统路径
current_dir = os.path.normpath(current_dir)
sys.path.append(current_dir)

# 加载.env文件中的环境变量
load_dotenv(dotenv_path=os.path.join(current_dir, ".env"))

# 加载项目配置文件
with open(os.path.join(current_dir, "config.json"), encoding="utf-8") as file:
    config = json.load(file)


In [ ]:
import qrcode
import qrcode.image.svg
from xml.etree import ElementTree as ET
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

# ====== 位置独立配置source参数 ======
DEFAULT_SOURCE = "homepage"  # 你可以在这里全局配置默认source

def add_source_to_url(url, default_source=None):
    """
    检查url中是否有source参数，如果没有则添加默认source
    """
    if default_source is None:
        default_source = DEFAULT_SOURCE
    parsed = urlparse(url)
    query = parse_qs(parsed.query)
    if "source" not in query:
        query["source"] = [default_source]
        new_query = urlencode(query, doseq=True)
        new_url = urlunparse((
            parsed.scheme,
            parsed.netloc,
            parsed.path,
            parsed.params,
            new_query,
            parsed.fragment
        ))
        return new_url
    else:
        return url

def generate_svg_qrcode(data, color="#000000", bg_color="#FFFFFF", source=None):
    # 检查并补充source参数
    data = add_source_to_url(data, default_source=source)
    print(data)
    # 创建二维码实例
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_M,
        box_size=3,
        border=1,
    )

    # 添加数据
    qr.add_data(data)
    qr.make(fit=True)

    # 生成SVG
    factory = qrcode.image.svg.SvgPathImage
    img = qr.make_image(image_factory=factory,
                       fill_color=color,
                       back_color=bg_color)

    # 转换为字符串并返回
    return img.to_string(encoding='unicode')

# 使用示例
qr_svg = generate_svg_qrcode(
    "https://otokonoizumi.github.io#projects",
    color="#FB7299",
    bg_color="#FFFFFF",
    source="weibo"
)
qr_svg